# 结构化规划代理

代理中的一个关键模式是规划能力。例如，ReAct使用结构化方法将输入分解为一组函数调用和思考，以便推理出最终的响应。

然而，将初始输入/任务分解为多个子任务可以使ReAct循环（或其他推理循环）更容易执行。

LlamaIndex中的`StructuredPlanningAgent`封装了任何代理工作者（ReAct、函数调用、抽象链等），并将初始输入分解为多个子任务。每个子任务由一个输入、预期结果以及应该首先完成的任何依赖子任务表示。

本笔记本将介绍此代理的高级和低级用法。

**注意：** 该代理利用了结构化输出和代理推理。因此，我们建议使用能力强大的LLM（OpenAI、Anthropic等），而开源的LLM可能需要进行提示工程或微调才能进行规划。


## 设置

为了创建计划，我们需要一套工具来在其上创建计划。在这里，我们使用一些经典的10k示例。


In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# 使用JSON模式的ollama
Settings.llm = OpenAI(
    model="gpt-4-turbo",
    temperature=0.1,
)
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool

# 加载文档，创建工具
lyft_documents = SimpleDirectoryReader(
    input_files=["./data/10k/lyft_2021.pdf"]
).load_data()
uber_documents = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

lyft_index = VectorStoreIndex.from_documents(lyft_documents)
uber_index = VectorStoreIndex.from_documents(uber_documents)

lyft_tool = QueryEngineTool.from_defaults(
    lyft_index.as_query_engine(),
    name="lyft_2021",
    description="用于询问有关Lyft 2021年10-K填报的问题。",
)

uber_tool = QueryEngineTool.from_defaults(
    uber_index.as_query_engine(),
    name="uber_2021",
    description="用于询问有关Uber 2021年10-K填报的问题。",
)

## 高级API

在本节中，我们将介绍用于创建和与结构化规划代理进行交流的高级API。

### 创建代理


In [ ]:
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)

# 为推理创建函数调用工作者
worker = FunctionCallingAgentWorker.from_tools(
    [lyft_tool, uber_tool], verbose=True
)

# 将工作者包装在顶层规划器中
agent = StructuredPlannerAgent(
    worker, tools=[lyft_tool, uber_tool], verbose=True
)

### 给予代理人一个复杂的任务


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
response = agent.chat(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

=== Initial plan ===
Extract Lyft Risk Factors:
Summarize the key risk factors from Lyft's 2021 10-K filing. -> A summary of the key risk factors for Lyft as outlined in their 2021 10-K filing.
deps: []


Extract Uber Risk Factors:
Summarize the key risk factors from Uber's 2021 10-K filing. -> A summary of the key risk factors for Uber as outlined in their 2021 10-K filing.
deps: []


Combine Risk Factors Summaries:
Combine the summaries of key risk factors for Lyft and Uber from their 2021 10-K filings into a comprehensive overview. -> A comprehensive summary of the key risk factors for both Lyft and Uber as outlined in their respective 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


> Running step a1428164-9f35-4a0e-9d6d-3328ebe490d4. Step input: Summarize the key risk factors from Lyft's 2021 10-K filing.
Added user message to memory: Summarize the key risk factors from Lyft's 2021 10-K filing.
> Running step 16a0d1f5-03ba-4b47-af0f-ec8758e6dda

In [ ]:
print(str(response))

assistant: The combined key risk factors from the 2021 10-K filings of Lyft and Uber highlight several shared and distinct challenges in the ridesharing and mobility industry:

1. **Pandemic and Economic Impact**: Both companies are significantly affected by the COVID-19 pandemic and broader economic conditions, which influence market demand and operational stability.

2. **Regulatory and Labor Issues**: A major concern is the classification of drivers as independent contractors versus employees, impacting labor costs and operational flexibility.

3. **Intense Competition**: They face fierce competition in the mobility, delivery, and logistics sectors, characterized by low entry barriers and aggressive, well-funded competitors.

4. **Operational and Financial Uncertainty**: Both companies deal with financial unpredictability and operational challenges, including managing insurance, claims, and maintaining profitability amidst high expenses.

5. **Technological and Service Evolution**: 

## 更改提示

`StructuredPlanningAgent` 有两个关键提示：
1. 初始规划提示
2. 规划细化提示

下面，我们将展示如何配置这些提示，以默认值作为示例。


In [ ]:
DEFAULT_INITIAL_PLAN_PROMPT = """\
一步一步地思考。给定一个任务和一组工具，创建一个全面的、端到端的计划来完成任务。
请记住，如果任务足够简单，就不需要将其分解为多个子任务。
计划应以满足整体任务的子任务结束。

可用的工具有：
{tools_str}

整体任务：{task}
"""

DEFAULT_PLAN_REFINE_PROMPT = """\
一步一步地思考。给定一个整体任务、一组工具和已完成的子任务，更新（如果需要）剩余的子任务，以便仍然可以完成整体任务。
计划应以满足整体任务的子任务结束。
如果剩余的子任务足够，可以跳过此步骤。

可用的工具有：
{tools_str}

整体任务：
{task}

已完成的子任务 + 输出：
{completed_outputs}

剩余的子任务：
{remaining_sub_tasks}
"""

In [ ]:
agent = StructuredPlannerAgent(
    worker,
    tools=[lyft_tool, uber_tool],
    initial_plan_prompt=DEFAULT_INITIAL_PLAN_PROMPT,
    plan_refine_prompt=DEFAULT_PLAN_REFINE_PROMPT,
    verbose=True,
)

## 低级API [高级]

在本节中，我们使用相同的代理，但是公开底层发生的步骤。

当您想要向人类公开底层计划、任务等，以便在运行时对其进行修改，或者用于调试和逐步运行事务时，这将非常有用。


### 创建代理


In [ ]:
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)

# 创建用于推理的react worker
worker = FunctionCallingAgentWorker.from_tools(
    [lyft_tool, uber_tool], verbose=True
)

# 将worker包装在顶层规划器中
agent = StructuredPlannerAgent(
    worker, tools=[lyft_tool, uber_tool], verbose=True
)

### 创建初始任务和计划


In [ ]:
plan_id = agent.create_plan(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

=== Initial plan ===
Extract Lyft Risk Factors:
Extract the key risk factors from Lyft's 2021 10-K filing. -> A detailed list of key risk factors for Lyft from its 2021 10-K filing.
deps: []


Extract Uber Risk Factors:
Extract the key risk factors from Uber's 2021 10-K filing. -> A detailed list of key risk factors for Uber from its 2021 10-K filing.
deps: []


Summarize Risk Factors:
Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings. -> A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']




### 检查初始任务和计划


In [ ]:
plan = agent.state.plan_dict[plan_id]

for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Extract Lyft Risk Factors =====
Expected output:  A detailed list of key risk factors for Lyft from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Extract Uber Risk Factors =====
Expected output:  A detailed list of key risk factors for Uber from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Summarize Risk Factors =====
Expected output:  A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
Dependencies:  ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


### 执行第一组任务

在这里，我们执行第一组任务，并确保它们的依赖关系得到满足。


In [ ]:
next_tasks = agent.state.get_next_sub_tasks(plan_id)

for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)


for sub_task in next_tasks:
    response = agent.run_task(sub_task.name)
    agent.mark_task_complete(plan_id, sub_task.name)

===== Sub Task Extract Lyft Risk Factors =====
Expected output:  A detailed list of key risk factors for Lyft from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Extract Uber Risk Factors =====
Expected output:  A detailed list of key risk factors for Uber from its 2021 10-K filing.
Dependencies:  []
> Running step 4f708bcd-3078-4138-897b-e7f643fc7f35. Step input: Extract the key risk factors from Lyft's 2021 10-K filing.
Added user message to memory: Extract the key risk factors from Lyft's 2021 10-K filing.
=== Calling Function ===
Calling function: lyft_2021 with args: {"input": "key risk factors"}
=== Function Output ===
Key risk factors for Lyft include general economic factors such as the impact of the COVID-19 pandemic, natural disasters, and macroeconomic conditions; operational factors such as limited operating history, competition, unpredictability of results, and the ability to attract and retain drivers and riders; and specific risks related to technology, such as a

如果我们愿意，甚至可以逐步执行每个任务。代码看起来会是这样的：

```python
# 按步骤执行每个任务

for sub_task in next_tasks:
    # 从状态中获取任务
    task = agent.state.get_task(sub_task.name)

    # 运行初始推理步骤
    step_output = agent.run_step(task.task_id)

    # 循环直到达到最后一步
    while not step_output.is_last:
        step_output = agent.run_step(task.task_id)
    
    # 完成响应并提交到内存
    agent.finalize_response(task.task_id, step_output=step_output)
```


### 检查是否完成

如果没有剩余任务，那么我们可以停止。否则，我们可以完善当前计划并继续。


In [ ]:
next_tasks = agent.get_next_tasks(plan_id)
print(len(next_tasks))

1


In [ ]:
for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task} =====")

===== Sub Task Summarize Risk Factors =====


### 优化计划

由于我们还有任务未完成，让我们优化我们的计划，以确保我们在正确的轨道上。


In [ ]:
# 优化计划
agent.refine_plan(
    "总结Lyft和Uber在2021年的10-K申报中的关键风险因素。",
    plan_id,
)

=== Refined plan ===
Summarize Risk Factors:
Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings. -> A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']




In [ ]:
plan = agent.state.plan_dict[plan_id]

for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Summarize Risk Factors =====
Expected output:  A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
Dependencies:  ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


### 循环直到完成

经过我们的计划完善，我们可以重复这个过程，直到没有更多的任务需要运行。


In [ ]:
import asyncio

while True:
    # 我们完成了吗？
    next_tasks = agent.get_next_tasks(plan_id)
    if len(next_tasks) == 0:
        break

    # 并发运行以获得更好的性能
    responses = await asyncio.gather(
        *[agent.arun_task(task_id) for task_id in next_tasks]
    )
    for task_id in next_tasks:
        agent.mark_task_complete(plan_id, task_id)

    # 优化计划
    await agent.arefine_plan(
        "总结Lyft和Uber在2021年的10-K申报中的关键风险因素。",
        plan_id,
    )

> Running step 70356e9a-98ee-49f5-b15f-e5a6b43381d0. Step input: Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings.
Added user message to memory: Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings.
=== LLM Response ===
The key risk factors for Lyft and Uber from their 2021 10-K filings highlight several areas of concern for both companies, with some overlapping and unique challenges:

### Common Risk Factors:
- **Economic and Market Conditions:** Both companies are affected by general economic factors such as macroeconomic conditions and the impact of the COVID-19 pandemic. Additionally, Uber faces specific market risks like interest rate risk, investment risk, and foreign currency risk due to its global operations.

### Lyft-Specific Risk Factors:
- **Operational Challenges:** Lyft faces risks related to its limited operating history, competition, unpr

最终，我们应该有一个单一的响应，这就是我们的最终答复。


In [ ]:
print(str(responses[-1]))

assistant: The key risk factors for Lyft and Uber from their 2021 10-K filings highlight several areas of concern for both companies, with some overlapping and unique challenges:

### Common Risk Factors:
- **Economic and Market Conditions:** Both companies are affected by general economic factors such as macroeconomic conditions and the impact of the COVID-19 pandemic. Additionally, Uber faces specific market risks like interest rate risk, investment risk, and foreign currency risk due to its global operations.

### Lyft-Specific Risk Factors:
- **Operational Challenges:** Lyft faces risks related to its limited operating history, competition, unpredictability of results, and the ability to attract and retain drivers and riders.
- **Technology and Insurance Risks:** Challenges include autonomous vehicle development, security breaches, reliance on third-party service providers, insurance coverage adequacy, and handling of auto-related insurance claims by third parties.
- **Regulatory a